# Day 19

link: https://adventofcode.com/2022/day/19

ข้อนี้ทำเอามึนไปเหมือนกัน

จากโจทย์เรามี resources อยู่ 4 ชนิด คือ ore, clay, obsidian และ geode แต่ละชนิดต้องใช้หุ่นยนต์เฉพาะของแต่ละชนิดในการ collect

เราสังเกตจาก input ว่ามันมี pattern ของการประกอบหุ่นยนต์ที่ตายตัว เพราะฉะนั้นเรา parse แค่ตัวเลข (ไม่ต้อง parse text)
แล้วจับใส่ array ซึ่งตัวเลขในตำแหน่ง (i, j) ก็คือจำนวน resources ชนิด j ที่ต้องใช้เพื่อสร้าง robot ชนิด i

In [1]:
input = IO.foreach('in19.txt').to_a.map(&:strip)

all_costs = input.map{|line|
  args = line.split(' ')
  [
    [args[6].to_i, 0, 0], 
    [args[12].to_i, 0, 0],
    [args[18].to_i, args[21].to_i, 0],
    [args[27].to_i, 0, args[30].to_i]
  ]
}
nil

เราใช้วิธี recursive backtracking ทีแรกลองคิดว่า ในทุกๆ time step เราจะมี 5 ทางเลือก คือเลือกประกอบหุ่นชนิดที่ 1, 2, 3, 4 หรือไม่ทำอะไร
แน่นอนว่าจริงๆ มันไม่ได้ทำได้ทุกทางเลือก ขึ้นกับว่า resources มีเพียงพอมั้ย แต่เราก็รู้สึกว่า search tree มันกว้างเกินไปอยู่ดี

ลองคิดต่อไปว่า สมมติใน time step = t เรามี resources พอที่จะสร้างหุ่นชนิดที่ 1 แต่เราเลือก "ไม่ทำอะไร" ใน step นี้ 
จากนั้นใน step t+1 เราไปเลือกประกอบหุ่นชนิดที่ 1 จะเห็นว่าการเลือกแบบนี้ไม่ optimal แน่ๆ คือถ้าจะประกอบหุ่นชนิดที่ 1 อยู่แล้ว
ก็ควรจะทำทันทีที่มี resources พอที่จะทำได้ เหตุผลของการ "ไม่ทำอะไร" มันจะฟังขึ้นก็ต่อเมื่อเป็นการรอเพื่อให้มี resources พอ
สำหรับหุ่นชนิดที่ตั้งใจไว้

ดังนั้นเราเปลี่ยนวิธีนิดหน่อย แทนที่จะมี 5 ทางเลือกในแต่ละ step เราจะเลือกกำหนดไปเลยว่า หุ่นตัวต่อไปจะเป็นชนิดไหน
ซึ่งการตัดสินใจนี้จะเกิดขึ้นหลังจากสร้างหุ่นเสร็จเท่านั้น ไม่ใช่ในทุก time step ดังนั้นจำนวนครั้งที่ต้องแตก branch จะลดลงไปมาก

ขุั้นตอนในแต่ละ step ก็จะเป็นประมาณนี้
1. เช็คว่ามี resources พอสำหรับสร้างหุ่น ชนิดที่กำหนดไว้หรือไม่ ถ้าพอก็หัก resources ไว้
2. เพิ่ม resources ตามจำนวนหุ่นที่มี
3. ในกรณีที่ resources ไม่พอ ก็ไป step ถัดไปเลย แต่ถ้าพอ ก็เพิ่มจำนวนหุ่น และเลือกหุ่นตัวต่อไป

ถึงตรงนี้เราลองดูแล้วก็ยังช้าเกินไปอยู่ดี ต้องหาทาง prune search tree ให้ได้อีก

เราไปสังเกตดู pattern ของจำนวน resources ที่ใช้ แล้วก็พบว่า ถึงแม้ว่า ore จะต้องใช้เป็น resource ของหุ่นทุกชนิด แต่ก็ใช้ในจำนวนไม่มาก
เทียบกับ clay และ obsidian ดังนั้นเรา assume ได้ว่าจำนวน ore robot ก็ไม่ควรจะต้องมีเยอะ
ตรงนี้เรา trial and error ไปๆ มาๆ แล้วก็พบว่าเราสามารถจำกัดจำนวน ore robot ไว้ที่ไม่เกิน 3 ตัวได้

อีกจุดนึงคือในจังหวะที่เลือกหุ่นที่จะสร้าง เราไปสร้าง priority ให้ geode กับ obsidian robots
คือจะเช็คแบบ greedy ว่า ถ้ามี resources พอสำหรับสร้าง geode robot ก็จะ fix ไปเลยว่าให้สร้างตัวนี้
ถ้าไม่พอ เราไปเช็คแบบเดียวกันกับ obsidian robot
ถ้ายังไม่ได้อีกถึงค่อยแตก branch ตรงนี้ช่วยลด search space ได้เยอะมาก

นอกจากนี้ยังมีการ bound search space คือตอนเริ่มแต่ละ time step เราเพิ่มการเช็คว่าใน best case scenario
เรายังมีหวังที่จะ improve result ได้หรือไม่ วิธีการคือ เราจะ assume ว่าเราสามารถสร้าง geode robot
ได้ในทุกๆ time step ที่เหลืออยู่ แล้วคำนวณดูว่าในกรณีนี้จะได้ geode เป็นจำนวนเท่าไหร่
ถ้ามันไม่ดีกว่าค่าที่ดีที่สุดที่เราเคยทำได้ ก็ไม่จำเป็นต้องหาใน branch นี้ต่อ

In [2]:
def can_build_with?(required, resources)
  3.times.all?{|i| resources[i] >= required[i]}
end

def tr(n) = n*(n+1)/2

def find_max_geodes(remain_steps, costs, resources, bots, next_target, best)
  
  return [resources[3], best].max if remain_steps == 0

  # assume we can build bot[3] every time step
  bound = resources[3] + bots[3] * remain_steps + tr(remain_steps - 1)
  return best if bound <= best

  req = costs[next_target]
  can_build = can_build_with?(req, resources)

  n_resources = 4.times.map{|i| resources[i] + bots[i] - (can_build && i < 3 ? req[i] : 0)}
  
  if can_build
    n_bots = 4.times.map{|i| i == next_target ? bots[i] + 1 : bots[i]}
    if can_build_with?(costs[3], n_resources)
      find_max_geodes(remain_steps - 1, costs, n_resources, n_bots, 3, best)
    elsif can_build_with?(costs[2], n_resources)
      find_max_geodes(remain_steps - 1, costs, n_resources, n_bots, 2, best)
    else
      min_target = bots[0] > 2 ? 1 : 0

      min_target.upto(3).each{|i|
        res = find_max_geodes(remain_steps - 1, costs, n_resources, n_bots, i, best)
        best = res if res > best
      }
      best
    end
  else
    find_max_geodes(remain_steps - 1, costs, n_resources, bots, next_target, best)
  end

end

nil

## Part 1

In [3]:
puts all_costs.each_with_index.map{|cs, i|
  v1 = find_max_geodes(24, cs, [0, 0, 0, 0], [1, 0, 0, 0], 0, 0)
  v2 = find_max_geodes(24, cs, [0, 0, 0, 0], [1, 0, 0, 0], 1, v1)
  v2 * (i + 1)
}.sum

1766


## Part 2

In [4]:
puts all_costs[0...3].map{|cs|
  v1 = find_max_geodes(32, cs, [0, 0, 0, 0], [1, 0, 0, 0], 0, 0)
  v2 = find_max_geodes(32, cs, [0, 0, 0, 0], [1, 0, 0, 0], 1, v1)
  v2
}.reduce(:*)

30780
